In [ ]:
from amuse.lab import *
from amuse.community.fi.interface import Fi
from amuse.couple.bridge import Bridge
from amuse.ext.orbital_elements import generate_binaries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# ============================================================
# 1) Helper functions
# ============================================================

def get_blackhole_binary():
    M1 = 100 | units.MSun
    M2 = 100 | units.MSun
    a_bh = 10 | units.AU
    e_bh = 0.0
    bh1, bh2 = generate_binaries(M1, M2, a_bh, e_bh, G=constants.G)
    bhs = Particles()
    bhs.add_particle(bh1)
    bhs.add_particle(bh2)
    bhs.move_to_center()
    return bhs

def make_uniform_gas_sphere(N, mass, radius, temperature=5e3 | units.K):
    x, y, z = np.random.uniform(-1, 1, (3, N))
    r = np.sqrt(x**2 + y**2 + z**2)
    keep = r <= 1
    x, y, z = x[keep], y[keep], z[keep]
    pos = np.column_stack((x, y, z))
    pos /= np.max(np.sqrt(np.sum(pos**2, axis=1)))  # normalize

    gas = Particles(len(pos))
    gas.mass = mass / len(gas)
    gas.position = radius * pos
    gas.velocity = [0, 0, 0] | units.m/units.s
    mean_molecular_weight = 0.6
    mh = 1.6735575e-27 | units.kg
    u = 1.5 * constants.kB * temperature / (mean_molecular_weight * mh)
    gas.u = u
    return gas

# ============================================================
# 2) Initialize systems
# ============================================================

bhs = get_blackhole_binary()
gas = make_uniform_gas_sphere(500, 1.0 | units.MSun, 1.0 | units.RSun)

# Move the gas closer for tidal stripping
gas.position += [100, 0, 0] | units.AU  
gas.velocity += [-200, 0, 0] | units.km/units.s 

# ============================================================
# 3) Create solvers
# ============================================================

converter_grav = nbody_system.nbody_to_si(1 | units.AU, 1 | units.MSun)
gravity = ph4(converter_grav)
gravity.particles.add_particles(bhs)
gravity.parameters.timestep_parameter = 0.03
gravity.parameters.epsilon_squared = (0.01 | units.AU)**2  # small softening

converter_hydro = nbody_system.nbody_to_si(1 | units.RSun, 1 | units.MSun)
hydro = Fi(converter_hydro, num_of_workers=1)
hydro.gas_particles.add_particles(gas)
hydro.parameters.use_hydro_flag = True
hydro.parameters.self_gravity_flag = True   # keep self-gravity
hydro.parameters.gamma = 5.0/3.0
hydro.parameters.timestep = 5e3 | units.s
hydro.parameters.artificial_viscosity_alpha = 0.5

# Short relaxation
hydro.evolve_model(0.2 | units.day)

# ============================================================
# 4) Couple via Bridge
# ============================================================

bridge = Bridge(use_threading=False)
bridge.add_system(gravity, (hydro,))
bridge.add_system(hydro, (gravity,))
bridge.timestep = 0.2 | units.day  # smaller step for stability

# ============================================================
# 5) Animation setup
# ============================================================

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X [AU]')
ax.set_ylabel('Y [AU]')
ax.set_zlabel('Z [AU]')
ax.set_xlim(-400, 400)
ax.set_ylim(-400, 400)
ax.set_zlim(-100, 100)
ax.set_title('BH Binary + Self-Gravitating Gas (80 days, stripped star)')

bh_dots, = ax.plot([], [], [], 'o', color='red', markersize=8, label='Black Holes')
gas_scatter = ax.scatter([], [], [], s=1, color='blue', alpha=0.5, label='Gas')
ax.legend()

# ============================================================
# 6) Animation update
# ============================================================
# stopping condition until star is stripped
t_end = 20 | units.day
dt = 0.5 | units.day
frames = int(t_end/dt)

def init():
    gas_scatter._offsets3d = ([], [], [])
    bh_dots.set_data([], [])
    bh_dots.set_3d_properties([])
    return bh_dots, gas_scatter

def update(frame):
    time = (frame+1) * dt
    bridge.evolve_model(time)

    bhs_pos = gravity.particles.position.value_in(units.AU)
    gas_pos = hydro.gas_particles.position.value_in(units.AU)

    bh_dots.set_data(bhs_pos[:,0], bhs_pos[:,1])
    bh_dots.set_3d_properties(bhs_pos[:,2])
    gas_scatter._offsets3d = (gas_pos[:,0], gas_pos[:,1], gas_pos[:,2])

    ax.set_title(f"t = {time.value_in(units.day):.1f} days")
    return bh_dots, gas_scatter

# ============================================================
# 7) Run animation
# ============================================================

anim = FuncAnimation(fig, update, frames=frames, init_func=init,
                     interval=100, blit=False, repeat=False)
anim.save('bridge_bh_gas_shoot_100.mp4', writer='ffmpeg', fps=20, dpi=150)
plt.show()

bridge.stop()



Roche limit - 10 solar mass bh - 1 solar mass star = 0.010 AU